

Disclaimer: This code should only be used as a reference and the responsibility of using this code lies solely on the user/users who is/ are using/executing it.

The library used is BeautifulSoup. This program scrapes Metacritics, IMDB and Rotten Tomatoes for movies genres. then save them to CSVs and SQLite. Finally, the correlation between the website on the genres is found out.


In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import numpy as np
import csv, sqlite3

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
no_movies = 50 # You can change this number to change to number of the movies scarped.
               # It can accept value from 0 to 100.

In [2]:
# make dictionary
def make_dictionary(movies, genres):
    dict_movie = {}
    for movie, gen in zip(movies, genres):
        dict_movie[movie] = gen
    return dict_movie

# Make the vector dictionary
def make_vector(dict_movie):
    genre_vect = {}
    for movie in dict_movie.keys():
        for i in dict_movie[movie]:
            i=i.lower()
            genre_vect[i] = genre_vect.get(i,0)+1
    return genre_vect

# Make Dataframe
def make_dataframe(dict_movie):
    largest = 0
    val = None
    df=None
    for i in dict_movie.keys():
        if largest<len(dict_movie[i]):
            largest = len(dict_movie[i])
            val = i

    edited_movie_names = []
    for i in dict_movie.keys():
        edited_movie_names.append("Movie Name "+str(i))

    df = pd.DataFrame(columns = edited_movie_names)
    df["Movie Name "+str(val)] = pd.Series(np.array(dict_movie[val]))
    for i in dict_movie.keys():
        df["Movie Name "+str(i)] = pd.Series(np.array(dict_movie[i]))
    return df

# makes a table of gebres for the movie in an sqlite database
def make_table(website, df):
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace(',', '')
    df.columns = df.columns.str.replace('.', '')
    df.columns = df.columns.str.replace(':', '')
    df.columns = df.columns.str.replace(')', '')
    df.columns = df.columns.str.replace('(', '')
    df.columns = df.columns.str.replace("'", '')
    df.columns = df.columns.str.replace("-", '')
    
    string =""
    k=0
    for i in list(df.columns):
        if k!=0:
            string+=", "
        k=1
        string +=i
        string +=" text"
    
    conn = sqlite3.connect('Genres') 
    cur = conn.cursor()
    cmd = "DROP TABLE IF EXISTS "+str(website)
    cur.execute(cmd)
    cmd = "CREATE TABLE "+ str(website) +"("+ string +");"
    cur.execute(cmd)
    df.to_sql(str(website), conn, if_exists='append', index = False)
    print(f"Table for {website} created")
    conn.close()
    

# 1. Metacritics

In [3]:
# The url to be entered is given below
print("You get 3 tries to make the url right! The right url is given in the markdown cell below.")
iteration = 0
while iteration<3:
    url = input('Enter url for Metacritics- ')
#     url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
    if url == "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc":
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'clamp-summary-wrap')

        movie_names = [] # list of first no_movies movies names
        movie_links = [] # list of first no_movies movies names to scrape genres
        movies = container[:no_movies] #here we get the top 50 movies we want
        
        # Scraping movie names
        for movie in movies:
            name = movie.find('h3').text
            movie_names.append(name)
    
        # Scraping individual movie links
        for movie in movies:
            tag = movie.find('a', class_ = 'title')
            link = tag.get('href', None)
            movie_links.append(link)

        movie_genre = {}
        final_raw_genres = []
        genres=[]
        
        # Scraping the genres of all the movies
        for link in movie_links:
            url = "https://www.metacritic.com/"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")
            genre_container = soup.find_all('div', class_ = 'genres')
            genre_container = genre_container[:]
            raw_genres = [span.text for span in genre_container]

            for i in raw_genres:
                temp = i.split(',')
                temp1 = [k.replace("\n","") for k in temp]
                temp1[0] = temp1[0][9:]
                for k in range(len(temp1)):
                    temp1[k]=temp1[k].strip().lower()
            final_raw_genres.append(temp1)
        
        # Dictionary with movies name as keys and corresponding genres as the values
        dict_movie_metacritics = make_dictionary(movie_names, final_raw_genres)
        # Dictionary with genre as keys and number of times the genre appears as the values
        genre_vect_metacritics = make_vector(dict_movie_metacritics)
        # Dataframa of genres with movie names as columns names
        df_metacritics = make_dataframe(dict_movie_metacritics)
        # Make CSV of genres
        df_metacritics.to_csv('Metacritics_Genres.csv',index=False)
        # Make a SQLite Databse with genres in a table
        make_table("Metacritics", df_metacritics)
        break
        
    if iteration<2:
        if iteration ==1:
            print("Wrong Url! You have only one try left! Try copying the right url again from the markdown cell below.")
        else:
            print("Wrong Url! You have ",2-iteration," tries left! Try copying the right url again from the markdown cell below.")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    iteration+=1


You get 3 tries to make the url right! The right url is given in the markdown cell below.
Enter url for Metacritics- https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc
Table for Metacritics created


### Metacritics
https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc

In [4]:
try:
    print("Genre dictionary for Metacritics:",dict_movie_metacritics)
except:
    print("Previous code blocks for Metacritics are not run correctly! Run them correctly first.")

Genre dictionary for Metacritics: {'Citizen Kane': ['drama', 'mystery'], 'The Godfather': ['drama', 'thriller', 'crime'], 'Rear Window': ['mystery', 'thriller'], 'Casablanca': ['drama', 'romance', 'war'], 'Boyhood': ['drama'], 'Three Colors: Red': ['drama', 'mystery', 'romance'], 'Vertigo': ['mystery', 'thriller', 'romance'], 'Notorious': ['drama', 'thriller', 'romance', 'film-noir'], "Singin' in the Rain": ['comedy', 'romance', 'musical'], 'City Lights': ['drama', 'comedy', 'romance'], 'Moonlight': ['drama'], 'Intolerance': ['drama', 'history'], 'Pinocchio': ['fantasy', 'comedy', 'animation', 'family', 'musical'], 'Touch of Evil': ['drama', 'thriller', 'crime', 'film-noir'], 'The Treasure of the Sierra Madre': ['adventure', 'drama', 'western'], "Pan's Labyrinth": ['drama', 'mystery', 'thriller', 'fantasy', 'war'], 'Some Like It Hot': ['comedy', 'romance'], 'North by Northwest': ['action', 'adventure', 'mystery', 'thriller'], 'Hoop Dreams': ['drama', 'sport', 'documentary'], 'Rashomon'

In [5]:
try:
    print("Dictionary of vectors for Metacritics:",genre_vect_metacritics)
except:
    print("Previous code blocks for Metacritics are not run correctly! Run them correctly first.")

Dictionary of vectors for Metacritics: {'drama': 39, 'mystery': 10, 'thriller': 13, 'crime': 4, 'romance': 11, 'war': 5, 'film-noir': 4, 'comedy': 9, 'musical': 3, 'history': 5, 'fantasy': 3, 'animation': 3, 'family': 3, 'adventure': 3, 'western': 2, 'action': 3, 'sport': 1, 'documentary': 1, 'biography': 2, 'horror': 3, 'music': 1}


In [6]:
try:
    conn = sqlite3.connect('Genres') 
    cur = conn.cursor()
    cur.execute("select Movie_Name_Citizen_Kane from Metacritics")
    for row in cur:
        if row !=(None,):
            print(row)
    conn.close()
except:
    print("Previous code blocks for Metacritics are not run correctly! Run them correctly first.")



('drama',)
('mystery',)


# 2. IMDB

In [7]:
# The url to be entered is given below
print("You get 3 tries to make the url right! The right url is given in the markdown cell below.")
iteration = 0
while iteration<3:
    url = input('Enter url for IMDB- ')
#     url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
    if url == "https://www.imdb.com/chart/top/?ref_=nv_mv_250":
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('td', class_ = 'titleColumn') 

        movie_names = []
        movie_links = []
        movies = container[:no_movies] #here we get the top 50 movies we want

        for movie in movies:
            name = movie.find('a').text
            movie_names.append(name)

        for movie in movies:
            tag = movie.find('a')
            link = tag.get('href', None)
            movie_links.append(link)

        container_genre = {}
        final_raw_genres = {}
        genres_list=[]
        container = None

        for link, movie in zip(movie_links, movie_names):
            url = "https://www.imdb.com/"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")
            container = soup.find_all('div', class_= "see-more inline canwrap")
            final_raw_genres[movie] = container[1].find_all('a')

        dict_movie_imdb = {}

        for i in final_raw_genres.keys():
            arr_genres = []
            for j in final_raw_genres[i]:
                arr_genres.append(j.text.strip().lower())
            dict_movie_imdb[i]=arr_genres

        genre_vect_imdb = make_vector(dict_movie_imdb)
        df_imdb = make_dataframe(dict_movie_imdb)

        df_imdb.to_csv('IMDB_Genres.csv',index=False)
        make_table("IMDB", df_imdb)
        break
        
    if iteration<2:
        if iteration ==1:
            print("Wrong Url! You have only one try left! Try copying the right url again from the markdown cell below.")
        else:
            print("Wrong Url! You have ",2-iteration," tries left! Try copying the right url again from the markdown cell below.")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    iteration+=1

You get 3 tries to make the url right! The right url is given in the markdown cell below.
Enter url for IMDB- https://www.imdb.com/chart/top/?ref_=nv_mv_250
Table for IMDB created


IMDB

https://www.imdb.com/chart/top/?ref_=nv_mv_250

In [8]:
try:
    print("Genre dictionary for IMDB:",dict_movie_imdb)
except:
    print("Previous code blocks for imdb are not run correctly! Run them correctly first.")

Genre dictionary for IMDB: {'The Shawshank Redemption': ['drama'], 'The Godfather': ['crime', 'drama'], 'The Godfather: Part II': ['crime', 'drama'], 'The Dark Knight': ['action', 'crime', 'drama', 'thriller'], '12 Angry Men': ['crime', 'drama'], "Schindler's List": ['biography', 'drama', 'history'], 'The Lord of the Rings: The Return of the King': ['action', 'adventure', 'drama', 'fantasy'], 'Pulp Fiction': ['crime', 'drama'], 'The Good, the Bad and the Ugly': ['western'], 'The Lord of the Rings: The Fellowship of the Ring': ['action', 'adventure', 'drama', 'fantasy'], 'Fight Club': ['drama'], 'Forrest Gump': ['drama', 'romance'], 'Inception': ['action', 'adventure', 'sci-fi', 'thriller'], 'The Lord of the Rings: The Two Towers': ['action', 'adventure', 'drama', 'fantasy'], 'Star Wars: Episode V - The Empire Strikes Back': ['action', 'adventure', 'fantasy', 'sci-fi'], 'The Matrix': ['action', 'sci-fi'], 'Goodfellas': ['biography', 'crime', 'drama'], "One Flew Over the Cuckoo's Nest": 

In [9]:
try:
    print("Dictionary of vectors for IMDB:",genre_vect_imdb)
except:
    print("Previous code blocks for imdb are not run correctly! Run them correctly first.")

Dictionary of vectors for IMDB: {'drama': 39, 'crime': 13, 'action': 13, 'thriller': 10, 'biography': 5, 'history': 2, 'adventure': 12, 'fantasy': 8, 'western': 2, 'romance': 5, 'sci-fi': 8, 'mystery': 7, 'comedy': 6, 'war': 5, 'family': 4, 'animation': 3, 'musical': 2, 'music': 2, 'horror': 1}


In [10]:
try:
    conn = sqlite3.connect('Genres') 
    cur = conn.cursor()
    cur.execute("select Movie_Name_The_Shawshank_Redemption from IMDB")
    for row in cur:
        if row !=(None,):
            print(row)
    conn.close()
except:
    print("Previous code blocks are for imdb not run correctly! Run them correctly first.")



('drama',)


# 3. Rotten Tomatoes



In [12]:
print("You get 3 tries to make the url right! The right url is given in the markdown cell below.")
iteration = 0
while iteration<3:
    url = input('Enter url for Rotten Tomatoes- ')
#     url = "https://www.rottentomatoes.com/top/bestofrt/"
    if url == "https://www.rottentomatoes.com/top/bestofrt/":
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.content, "html.parser")
        container = soup.find_all('a', class_="unstyled articleLink")
        container = container[43:-2]
        movie_names = []
        movie_links = []
        movies = container[:no_movies] #here we get the top 50 movies we want

        for movie in movies:
            movie=str(movie)
            start = [x for x, v in enumerate(movie) if v == '>'][0]
            end = [x for x, v in enumerate(movie) if v == '<'][1]
            movie_names.append(movie[start+2:end].strip())

        for movie in movies:
            link = movie.get('href', None)
            movie_links.append(link)

        movie_genre = {}
        final_raw_genres = []
        genres=[]

        for link in movie_links:
            url = "https://www.rottentomatoes.com"+link
            page = requests.get(url, headers = headers)
            soup = BeautifulSoup(page.content, "html.parser")

            genre_container = soup.find_all('div', class_="meta-value genre")
            raw_genres = [span.text for span in genre_container]

            temp_genre = []
            for i in raw_genres:
                temp= i.split(',')
            for i in temp:
                temp_genre.append(i.strip())
            final_raw_genres.append(temp_genre)

        dict_movie_rotten_tomatoes = make_dictionary(movie_names, final_raw_genres)
        genre_vect_rotten_tomatoes = make_vector(dict_movie_rotten_tomatoes)
        df_rotten_tomatoes = make_dataframe(dict_movie_rotten_tomatoes)

        df_rotten_tomatoes.to_csv('Rotten_Tomatoes_Genres.csv',index=False)
        make_table("Rotten_Tomatoes", df_rotten_tomatoes)
        break
        
    if iteration<2:
        if iteration ==1:
            print("Wrong Url! You have only one try left! Try copying the right url again from the markdown cell below.")
        else:
            print("Wrong Url! You have ",2-iteration," tries left! Try copying the right url again from the markdown cell below.")
    else:
        print("You have exceeded your number of tries! Exiting! Goodbye!")
        break
    iteration+=1

You get 3 tries to make the url right! The right url is given in the markdown cell below.
Enter url for Rotten Tomatoes- https://www.rottentomatoes.com/top/bestofrt/
Table for Rotten_Tomatoes created


### Rotten Tomatoes
https://www.rottentomatoes.com/top/bestofrt/

In [13]:
try:
    print("Genre dictionary for Rotten Tomatoes:",dict_movie_rotten_tomatoes)
except:
    print("Previous code blocks for Rotten Tomatoes are not run correctly! Run them correctly first.")



Genre dictionary for Rotten Tomatoes: {'Black Panther (2018)': ['fantasy', 'action', 'adventure'], 'Avengers: Endgame (2019)': ['fantasy', 'adventure', 'sci fi', 'action'], 'Us (2019)': ['horror', 'mystery and thriller'], 'Toy Story 4 (2019)': ['fantasy', 'kids and family', 'animation', 'adventure', 'comedy'], 'Lady Bird (2017)': ['drama', 'comedy'], 'Citizen Kane (1941)': ['drama'], 'Mission: Impossible - Fallout (2018)': ['adventure', 'mystery and thriller', 'action'], 'The Wizard of Oz (1939)': ['musical', 'kids and family', 'fantasy'], 'BlacKkKlansman (2018)': ['drama', 'comedy', 'crime'], 'Get Out (2017)': ['horror', 'mystery and thriller', 'comedy'], 'The Irishman (2019)': ['drama', 'crime'], 'Mad Max: Fury Road (2015)': ['adventure', 'action'], 'Spider-Man: Into the Spider-Verse (2018)': ['fantasy', 'comedy', 'kids and family', 'animation', 'adventure', 'action'], 'Casablanca (1942)': ['drama'], 'Moonlight (2016)': ['drama'], 'A Star Is Born (2018)': ['drama', 'music', 'romance'

In [14]:
try:
    print("Dictionary of vectors for Rotten Tomatoes:",genre_vect_rotten_tomatoes)
except:
    print("Previous code blocks  for Rotten Tomatoes are not run correctly! Run them correctly first.")

Dictionary of vectors for Rotten Tomatoes: {'fantasy': 13, 'action': 10, 'adventure': 14, 'sci fi': 5, 'horror': 4, 'mystery and thriller': 7, 'kids and family': 8, 'animation': 5, 'comedy': 18, 'drama': 21, 'musical': 2, 'crime': 5, 'music': 2, 'romance': 4, 'war': 2, 'history': 2}


In [15]:
try:
    conn = sqlite3.connect('Genres') 
    cur = conn.cursor()
    cur.execute("select Movie_Name_Black_Panther_2018 from Rotten_Tomatoes")
    for row in cur:
        if row !=(None,):
            print(row)
    conn.close()
except:
    print("Previous code blocks  for Rotten Tomatoes are not run correctly! Run them correctly first.")



('fantasy',)
('action',)
('adventure',)


Calculation

In [16]:
denominator_metacritics = 0
denominator_imdb = 0
denominator_rotten = 0
try:
    # Finding denominators
    for i in genre_vect_metacritics.keys():
        denominator_metacritics+=(genre_vect_metacritics.get(i))**2
    denominator_metacritics = (denominator_metacritics)**(1/2)

    for i in genre_vect_imdb.keys():
        denominator_imdb+=(genre_vect_imdb.get(i))**2
    denominator_imdb = (denominator_imdb)**(1/2)

    for i in genre_vect_rotten_tomatoes.keys():
        denominator_rotten+=(genre_vect_rotten_tomatoes.get(i))**2
    denominator_rotten = (denominator_rotten)**(1/2)

    # corelation between metacritics and imdb
    numerator_meta_imdb = 0
    for i in genre_vect_metacritics.keys():
        try:
            numerator_meta_imdb+= genre_vect_metacritics[i]*genre_vect_imdb[i]
        except:
            numerator_meta_imdb+=0

    # corelation between metacritics and rotten tomatoes
    numerator_meta_rotten = 0
    for i in genre_vect_metacritics.keys():
        try:
            numerator_meta_rotten+= genre_vect_metacritics[i]*genre_vect_rotten_tomatoes[i]
        except:
            numerator_meta_rotten+=0

    # corelation between imdb and rotten tomatoes
    numerator_imdb_rotten = 0
    for i in genre_vect_metacritics.keys():
        try:
            numerator_imdb_rotten+= genre_vect_imdb[i]*genre_vect_rotten_tomatoes[i]
        except:
            numerator_imdb_rotten+=0

except:
    print("Previous code blocks are not run correctly! Run them correctly first.")

print("Correlation Between Metacritics and IMDB",numerator_meta_imdb/(denominator_metacritics*denominator_imdb))
print("Correlation Between Metacritics and Rottan Tomatoes",numerator_meta_rotten/(denominator_metacritics*denominator_rotten))
print("Correlation Between IMDB and rotten",numerator_imdb_rotten/(denominator_imdb*denominator_rotten))

Correlation Between Metacritics and IMDB 0.9019873939979214
Correlation Between Metacritics and Rottan Tomatoes 0.6824329998331107
Correlation Between IMDB and rotten 0.7704153969743939
